# How to read and process the Wikdata dump file.

In [ ]:
import sys
sys.path.append('../src')

import os
os.environ["LANGUAGE"] = 'en' # Specify the language of the entity labels, description, and aliases.

from wikidataDumpReader import WikidataDumpReader
from wikidataDB import WikidataID, WikidataEntity, Session
from sqlalchemy import select
from multiprocessing import Manager
from tqdm import tqdm
import time
import os

#### Reading the Wikidata dump ZIP file and saving the IDs of entities and properties to SQLite (Only the ones connected to the English Wikipedia)

In [ ]:
FILEPATH = os.getenv("FILEPATH", '../data/Wikidata/latest-all.json.bz2')
BATCH_SIZE = int(os.getenv("BATCH_SIZE", 10000))
PUSH_SIZE = int(os.getenv("PUSH_SIZE", 20000))
QUEUE_SIZE = int(os.getenv("QUEUE_SIZE", 15000))
NUM_PROCESSES = int(os.getenv("NUM_PROCESSES", 4))
SKIPLINES = int(os.getenv("SKIPLINES", 0))
LANGUAGE = os.getenv("LANGUAGE", 'en')

In [ ]:
def save_ids_to_sqlite(item, bulk_ids, sqlitDBlock):
    if (item is not None) and WikidataID.is_in_wikipedia(item, language=LANGUAGE):
        ids = WikidataID.extract_entity_ids(item, language=LANGUAGE)
        bulk_ids.extend(ids)

        with sqlitDBlock:
            if len(bulk_ids) > PUSH_SIZE:
                worked = WikidataID.add_bulk_ids(list(bulk_ids[:PUSH_SIZE]))
                if worked:
                    del bulk_ids[:PUSH_SIZE]

multiprocess_manager = Manager()
sqlitDBlock = multiprocess_manager.Lock()
bulk_ids = multiprocess_manager.list()

wikidata = WikidataDumpReader(FILEPATH, num_processes=NUM_PROCESSES, batch_size=BATCH_SIZE, queue_size=QUEUE_SIZE, skiplines=SKIPLINES)
wikidata.run(lambda item: save_ids_to_sqlite(item, bulk_ids, sqlitDBlock), max_iterations=None, verbose=True)

while len(bulk_ids) > 0:
    worked = WikidataID.add_bulk_ids(list(bulk_ids))
    if worked:
        bulk_ids[:] = []
    else:
        time.sleep(1)

### Adding entities (label, description, claims, and aliases) of IDs found in WikidataID to WikidataEntity

In [ ]:
FILEPATH = os.getenv("FILEPATH", '../data/Wikidata/latest-all.json.bz2')
BATCH_SIZE = int(os.getenv("BATCH_SIZE", 1000))
PUSH_SIZE = int(os.getenv("PUSH_SIZE", 2000))
QUEUE_SIZE = int(os.getenv("QUEUE_SIZE", 1500))
NUM_PROCESSES = int(os.getenv("NUM_PROCESSES", 8))
SKIPLINES = int(os.getenv("SKIPLINES", 0))
LANGUAGE = os.getenv("LANGUAGE", 'en')

In [ ]:
def save_entities_to_sqlite(item, data_batch, sqlitDBlock):
    if (item is not None) and WikidataID.get_id(item['id']):
        item = WikidataEntity.normalise_item(item, language=LANGUAGE)
        data_batch.append(item)

        with sqlitDBlock:
            if len(data_batch) > PUSH_SIZE:
                worked = WikidataEntity.add_bulk_entities(list(data_batch[:PUSH_SIZE]))
                if worked:
                    del data_batch[:PUSH_SIZE]

multiprocess_manager = Manager()
sqlitDBlock = multiprocess_manager.Lock()
data_batch = multiprocess_manager.list()

wikidata = WikidataDumpReader(FILEPATH, num_processes=NUM_PROCESSES, batch_size=BATCH_SIZE, queue_size=QUEUE_SIZE, skiplines=SKIPLINES)
wikidata.run(lambda item: save_entities_to_sqlite(item, bulk_ids, sqlitDBlock), max_iterations=None, verbose=True)

while len(data_batch) > 0:
    worked = WikidataEntity.add_bulk_entities(list(data_batch))
    if worked:
        data_batch[:] = []
    else:
        time.sleep(1)

### Find IDs that are in WikidataID but not in WikidataEntity

In [ ]:
with Session() as session:
    result = session.execute(
        select(WikidataID.id)
        .outerjoin(WikidataEntity, WikidataID.id == WikidataEntity.id)
        .filter(WikidataEntity.id == None)
        .filter(WikidataID.in_wikipedia == True)
    )
    missing_ids = set(result.scalars().all())

print(len(missing_ids))

### Find IDs that are not in WikidataEntity but are in the claims, qualifiers, and quantity units of entities connected to Wikipedia

In [ ]:
def get_missing_entities(session, ids):
    existing_entities = session.query(WikidataEntity.id).filter(WikidataEntity.id.in_(ids)).all()
    existing_ids = {entity.id for entity in existing_entities}
    return set(ids) - existing_ids

with Session() as session:
    entities = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).yield_per(100000)

    progressbar = tqdm(total=9203531)
    found = False
    missing_ids = set()

    batch_size = 10000
    ids_to_check = set()

    for entity in entities:
        progressbar.update(1)
        for pid, claim in entity.claims.items():
            ids_to_check.add(pid)
            for c in claim:
                if ('datavalue' in c['mainsnak']):
                    if ((c['mainsnak']['datatype'] == 'wikibase-item') or (c['mainsnak']['datatype'] == 'wikibase-property')):
                        id = c['mainsnak']['datavalue']['value']['id']
                        ids_to_check.add(id)
                    elif (c['mainsnak']['datatype'] == 'quantity') and (c['mainsnak']['datavalue']['value']['unit'] != '1'):
                        id = c['mainsnak']['datavalue']['value']['unit'].rsplit('/', 1)[1]
                        ids_to_check.add(id)

                if 'qualifiers' in c:
                    for pid, qualifier in c['qualifiers'].items():
                        ids_to_check.add(pid)
                        for q in qualifier:
                            if ('datavalue' in q):
                                if ((q['datatype'] == 'wikibase-item') or (q['datatype'] == 'wikibase-property')):
                                    id = q['datavalue']['value']['id']
                                    ids_to_check.add(id)
                                elif (q['datatype'] == 'quantity') and (q['datavalue']['value']['unit'] != '1'):
                                    id = q['datavalue']['value']['unit'].rsplit('/', 1)[1]
                                    ids_to_check.add(id)

        if len(ids_to_check) >= batch_size:
            missing_ids.update(get_missing_entities(session, ids_to_check))
            ids_to_check.clear()

        if progressbar.n % 1000 == 0:
            progressbar.set_description(f"Missing IDs: {len(missing_ids)}")

    if ids_to_check:
        missing_ids.update(get_missing_entities(session, ids_to_check))

    progressbar.close()